In [2]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
import torch
import sympy

In [3]:
primelist=[]
for i in range(1,1001):
    primelist.append(sympy.prime(i))
x1=torch.tensor(primelist)[:,np.newaxis]
y1=torch.ones(1000,1)
print(np.shape(x1),np.shape(y1))

torch.Size([1000, 1]) torch.Size([1000, 1])


In [4]:
nonprimelist=[]
for i in range(1,100000):
    if not sympy.isprime(i):
        nonprimelist.append(i)
    if len(nonprimelist)==1000:
        break
x2=torch.tensor(nonprimelist)[:,np.newaxis]
y2=torch.zeros(1000,1)
print(np.shape(x2),np.shape(y2))

torch.Size([1000, 1]) torch.Size([1000, 1])


In [5]:
x=torch.cat((x1,x2),dim=0).type(torch.FloatTensor)
y=torch.cat((y1,y2),dim=0).squeeze().long()
x,y=Variable(x),Variable(y)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.layer1=nn.Sequential(nn.Linear(1,64),nn.Dropout(p=0.1),nn.LeakyReLU())
        self.layer2=nn.Sequential(nn.Linear(64,64),nn.Dropout(p=0.1),nn.LeakyReLU())
        self.layer3=nn.Sequential(nn.Linear(64,2),nn.LeakyReLU())
    def forward(self,x):
        #x=x.view(x.size()[0],-1)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        return x
net=Net()
print(net)

Net(
  (layer1): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (layer2): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (layer3): Sequential(
    (0): Linear(in_features=64, out_features=2, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
)


In [7]:
optimizer=optim.Adam(net.parameters(),lr=1e-2)
loss_func=torch.nn.CrossEntropyLoss()

for i in range(10001):
    out=net(x)
    loss=loss_func(out,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%400==0:
        prediction=torch.max(nn.functional.softmax(out,dim=1),1)[1]
        pred_y=prediction.data.numpy().squeeze()
        target_y=y.data.numpy()
        auc=(sum(pred_y==target_y))/2000
        print('epoch: {}, loss: {:4f}, auc: {:.4f}'.format(i,loss,auc))

epoch: 0, loss: 1.246590, auc: 0.4980
epoch: 400, loss: 0.537577, auc: 0.5175
epoch: 800, loss: 0.529179, auc: 0.5465
epoch: 1200, loss: 0.520550, auc: 0.5655
epoch: 1600, loss: 0.505802, auc: 0.6040
epoch: 2000, loss: 0.366838, auc: 0.8550
epoch: 2400, loss: 0.308962, auc: 0.8840
epoch: 2800, loss: 0.302073, auc: 0.8845
epoch: 3200, loss: 0.290500, auc: 0.8925
epoch: 3600, loss: 0.290490, auc: 0.8895
epoch: 4000, loss: 0.295206, auc: 0.8805
epoch: 4400, loss: 0.282315, auc: 0.8950
epoch: 4800, loss: 0.283334, auc: 0.8940
epoch: 5200, loss: 0.282818, auc: 0.8940
epoch: 5600, loss: 0.277425, auc: 0.8945
epoch: 6000, loss: 0.285728, auc: 0.8900
epoch: 6400, loss: 0.276010, auc: 0.8975
epoch: 6800, loss: 0.287751, auc: 0.8860
epoch: 7200, loss: 0.271791, auc: 0.8985
epoch: 7600, loss: 0.280800, auc: 0.8910
epoch: 8000, loss: 0.274130, auc: 0.8995
epoch: 8400, loss: 0.271387, auc: 0.9015
epoch: 8800, loss: 0.277034, auc: 0.8940
epoch: 9200, loss: 0.277774, auc: 0.8955
epoch: 9600, loss: 0.